In [ ]:
!pip install -U openai-whisper
!pip install python-docx

In [7]:
import whisper
import os
from glob import glob
from docx import Document
from datetime import timedelta

Exception ignored in: <function FFMPEG_AudioWriter.__del__ at 0x00000272E7892160>
Traceback (most recent call last):
  File "c:\Users\linoa\AppData\Local\R-MINI~1\envs\whs\lib\site-packages\moviepy\audio\io\ffmpeg_audiowriter.py", line 132, in __del__
    self.close()
  File "c:\Users\linoa\AppData\Local\R-MINI~1\envs\whs\lib\site-packages\moviepy\audio\io\ffmpeg_audiowriter.py", line 121, in close
    self.proc.stdin.close()
OSError: [Errno 22] Invalid argument


In [8]:
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large']

In [9]:
model = whisper.load_model("large-v2")

In [10]:
def parse_segments(segments):
    text = []
    for segment in segments:
        text.append(segment['text'].lstrip())
        text.append('\n')

    return ''.join(text)

def save_srt(result, file_name):
    outfile, _ = os.path.splitext(file_name)
    segments = result['segments']
    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] is ' ' else text}\n\n"

        srtFilename = os.path.join("videos", f"{outfile}.srt")
        with open(srtFilename, 'a', encoding='utf-8') as srtFile:
            srtFile.write(segment)


In [11]:
for file_name in glob(r'videos/*.mp3'):
    print(file_name)
    outfile, _ = os.path.splitext(file_name)
    result = model.transcribe(file_name)

    document = Document()
    document.add_paragraph(parse_segments(result['segments']))
    document.save(outfile + '.docx')


videos\ATC audio.mp3


# Output SRT for subtitles


FileNotFoundError: [Errno 2] No such file or directory: 'SrtFiles\\VIDEO_FILENAME.srt'

# MP4 to MP3

In [4]:
!pip install moviepy

In [6]:
from moviepy.editor import *
video = VideoFileClip('c:\\Users\\linoa\\Documents\\Code\\whisper_subtitles\\./videos/Pilot starts takeoff roll without clearance gets yelled at (ATC audio)  Happy Landings.mp4')
video.audio.write_audiofile('c:\\Users\\linoa\\Documents\\Code\\whisper_subtitles\\./videos/ATC audio.mp3')

                                                                  
chunk:   1%|          | 22/3634 [00:33<1:30:31,  1.50s/it, now=None]

MoviePy - Writing audio in c:\Users\linoa\Documents\Code\whisper_subtitles\./videos/ATC audio.mp3





















                                                                    
chunk:   1%|          | 22/3634 [00:34<1:35:19,  1.58s/it, now=None]

MoviePy - Done.


# Add subtitles to mp4

In [6]:


from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import TextClip, CompositeVideoClip

# Load video
video = VideoFileClip("videos/A 10 Close Air Support.mp4")

# Create a TextClip (subtitle)
subtitle = TextClip("Your subtitle here", fontsize=24, color='white')

# Position the subtitle at the bottom and for a duration of 10 seconds
subtitle = subtitle.set_position(('center', 'bottom')).set_duration(10)

# Overlay subtitle on video
final_video = CompositeVideoClip([video, subtitle])

# Write to a file
final_video.write_videofile("output_video.mp4")

ImportError: cannot import name 'TextClip' from 'moviepy.video.fx.all' (c:\Users\linoa\AppData\Local\R-MINI~1\envs\whs\lib\site-packages\moviepy\video\fx\all\__init__.py)

In [17]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import textwrap

# Load the video
video = VideoFileClip("videos/A 10 Close Air Support.mp4")

# Load the transcript file
transcript = result['segments']

# Create a subtitles clip
txt_clips = []
for text in transcript:
  subtitle = text['text']
  subtitle = "\n".join(textwrap.wrap(subtitle, 50))
  txt_clip = TextClip(subtitle,fontsize = 12,  font="Amiri-Bold", kerning=1, bg_color = 'black', color='white')
  txt_clip = txt_clip.set_start(text['start'])
  txt_clip = txt_clip.set_position((0.2,0.8), relative=True).set_duration(text['end']-text['start'])
  txt_clips.append(txt_clip)  
  

# concatenate the video and subtitles
final_video = CompositeVideoClip([video]+txt_clips)

# Save the final video
final_video.write_videofile("videos/output.mp4")

OSError: MoviePy Error: creation of None failed because of the following error:

[WinError 2] The system cannot find the file specified.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

# Download videos from Youtube for testing 

In [15]:
!pip install pytube

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.6 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 763.8 kB/s eta 0:00:00


In [2]:
from pytube import YouTube

link = "https://www.youtube.com/watch?v=jdBzyAURWEI"
yt = YouTube(link)

# Download the video with the highest resolution and file type (usually mp4)
video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
video.download('./videos/')

'c:\\Users\\linoa\\Documents\\Code\\whisper_subtitles\\./videos/Pilot starts takeoff roll without clearance gets yelled at (ATC audio)  Happy Landings.mp4'